In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
%matplotlib inline
# must be in python3 pygrib
import time
import multiprocessing as mp

In [3]:
def precintcon_ln_a(X, Y):
    N = len(X)
    if (len(X) == len(Y)):
        a=(sum(X**2)*sum(np.log(Y)))
        Z=((sum(X**2)*sum(np.log(Y))) + (sum(X)*sum(X*np.log(X)))-(sum(X**2)*sum(np.log(X))) -\
        (sum(X)*sum(X*np.log(Y))))/(N*sum(X**2)-(sum(X)**2))#!!!
    else:
        print("X and Y with differents lengths!")
    return Z

def precintcon_b(X, Y):
    N=len(X)
    if (len(X) == len(Y)):
        Z=(N * sum(X * np.log(Y))+ \
        sum(X) * sum(np.log(X))- \
        N * sum(X * np.log(X)) - \
        sum(X) * sum(np.log(Y))) \
        /(N * sum(X**2) - sum(X)**2)/100  #!!!!!   100   
    else:
        print("X and Y have differents lengths!")
    return Z

def p_sum_p(o):
    i = o[:,0]
    f = o[:,1]
    n = o[:,2]
    m = (o[:,0] + o[:,1] + 0.1)/2
    #print(len(m))
    x = m[0] * n[0]
    #print(x)
    #print(len(i))
    #print(len(f))
    #print(len(n))
    for k in range(1,len(i)):#!!!!!!!1
        #print(k)
        x = np.append(x, m[k] * n[k])
        #print(x)
    y = x[0]/sum(x)
    #print(y)
    #print(len(x))
    for k in range(2,len(x)+1):
        y = np.append(y, sum(x[0:k])/sum(x))
    return(y)

def p_sum_n(o):
    x = o[0]
    y=sum(o)
    #print(y)
    #print(len(y))
    for i in range(2,len(o)+1):
        #x = pd.concat(x, sum(o[0:i])) #np.r_
        #print(sum(o[0:i]))
        x = np.append(x, sum(o[0:i]))
        #print(x)
        n=x/y
    return(n)

def precintcon_class(v,interval):
    result=[0,0,0]
    #s=0
    #python-Error Message: 'float' object cannot be interpreted as an integer
    #print(v)
    #print(max(v))
    #print(math.ceil(max(v)))
    for i in range(0,math.ceil(max(v)+interval),interval):
        #initial = i # 
        initial = i + 0.1 if i == 0  else i
        final   = i+interval-0.1
        s = sum((v[(v>= initial)& (v<=final)]).value_counts())#!!!!!!!!!!!!!!!!!!!!! no table
        #s += sum(v[(v>= initial)& (v<=final)])
        #print(initial, final, s)
        #if not s.empty: 
        if s>0 :
            #result= np.append(initial, final, s)
            result= np.row_stack((result, [initial, final, s]))
    #print(result)
    return result[1:,:]#!!!1
def ci_pre(X,Z):
    stid=X["Stid"]
    rain=X["rain"]
    Height=Z["Height"]
    Lon=Z["Lon"]
    Lat=Z["Lat"]
    Date=Z["DATE"]
    y=precintcon_class(rain,1)
    #print(y)
    sum_p=p_sum_p(y)
    sum_n=p_sum_n(y[:,2])
    a= np.exp(precintcon_ln_a(sum_n, sum_p))
    b= precintcon_b(sum_n, sum_p)
    A=(a/b)*(np.exp(b*100) * (100 - (1/b)))
    S= 5000-A
    ci= 2*S/10000
    cid=[Date.iloc[0],stid.iloc[0],Height.iloc[0],Lat.iloc[0],Lon.iloc[0],ci]
    #cidd=np.array(cid)
    #cidd=cid[0][0:2]
    cipd=pd.DataFrame(np.array([cid]),columns=["DATE","Stid","Height","Lat","Lon","ci"]) 
    return cipd

In [ ]:
df = pd.read_csv('china_pre-before2000.csv', sep=',',skiprows=0,header=0,na_values=[32766.0],low_memory=False)
df=df.dropna()
df[['rain','rain208','rain820']]=df[['rain','rain208','rain820']].applymap( lambda x: x if x<30000 else (x-30000 if x<31000 else (x-31000 if x<32000 else(x-32000 if x<32700 else x-32700))))

In [ ]:
times=pd.to_datetime(df['Time'],format='%Y-%m-%d')
from datetime import datetime
df['DATE'] = pd.to_datetime(df['Time'])
df['DATE'] = [datetime.strftime(x,'%Y') for x in df['DATE']]

In [ ]:
df.to_csv("use_pre-before2000-Y.csv",index=False)

In [4]:
df = pd.read_csv('use_pre-before2000-Y.csv', sep=',',skiprows=0,header=0,na_values=[32766.0],low_memory=False)
df=df.dropna()

In [5]:
df.head()

,Time,Stid,Lat,Lon,Height,rain,rain208,rain820,DATE
0,1951-01-01,50527,4913,11945,6766,0.0,0.0,0.0,1951
1,1951-01-01,56571,2753,10218,15907,0.0,0.0,0.0,1951
2,1951-01-01,58634,2840,11815,936,0.0,0.0,0.0,1951
3,1951-01-01,56492,2849,10432,2750,0.0,0.0,0.0,1951
4,1951-01-01,56386,2930,10345,3715,0.0,0.0,0.0,1951


In [6]:
def job(x):
    #print(x)
    ci=ci_pre(bf[x],al[x])
    #print(ci,x)
    return ci

In [6]:
if __name__ == '__main__':    
    cf=df[df["DATE"]==1952]
    #print(cf.head())
    stid=cf.Stid.drop_duplicates()
    #print(stid)
    bf=[]
    al=[]
    for id in stid:
        #af=df[["Stid","rain","Height","Lat","Lon","DATE"]][df["Stid"]==id]
        af=cf[["Stid","rain"]][cf["Stid"]==id]
        ef=cf[["Height","Lat","Lon","DATE"]][cf["Stid"]==id]
        a=ef.iloc[[0]]
        print(a)
        #af=cf[["Stid","rain"]][cf["Stid"]==id]
        if any(af["rain"]):#如果都为空、0、false，则返回false，如果不都为空、0、false，则返回true。
            bf.append(af)
            al.append(a)
                # TypeError: list indices must be integers or slices, not tuple 53602 is all 0.000
                     #for 1951Y bf[147] 56029 all is 66.0
    print(len(bf))
    #print(bf)
    pool = mp.Pool(processes=8)
    multi_res =[pool.apply_async(job, (i,)) for i in range(len(bf))]
    pool.close()
    pool.join()#调用join之前，先调用close函数，否则会出错。执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    #name=['Stid','rain']
    #test=pd.DataFrame(columns=name,data=mres)#数据有三列，列名分别为one,two,three
    #test.to_csv('testcsv231.csv',index=False)
    #test.to_csv("ci_"+str(i)+".csv",index=False,sep=',')
    output= pd.DataFrame()
    for i in range(len(bf)):
        try:
            result0=multi_res[i].get()
            print(result0)
        #cipd=pd.DataFrame(np.array([result0]),columns=["DATE","Stid","Height","Lat","Lon","ci"])
       # output.append(result0)
            output=pd.concat([output, result0], axis=0)
        except Exception as e:
            print("error")
    output.to_csv("ci_"+str(year)+".csv",index=False,sep=',')

       Height   Lat    Lon  DATE
37971     206  3256  11722  1952
       Height   Lat    Lon  DATE
37972    4050  2527  10646  1952
       Height   Lat    Lon  DATE
37973    1780  2515  11010  1952
       Height   Lat    Lon  DATE
37974  114050  3636  10439  1952
       Height   Lat    Lon  DATE
37975     560  2615  11420  1952
       Height   Lat    Lon  DATE
37976   15250  3805  10255  1952
       Height   Lat    Lon  DATE
37977     990  2550  11450  1952
       Height   Lat    Lon  DATE
37978     920  2656  11235  1952
       Height   Lat    Lon  DATE
37979     720  2705  11455  1952
       Height   Lat    Lon  DATE
37980    1640  2622  11131  1952
       Height   Lat    Lon  DATE
37981  109000  3636  10932  1952
       Height   Lat    Lon  DATE
37982  115500  3855  10035  1952
       Height   Lat    Lon  DATE
37983    6000  2738  10857  1952
       Height   Lat    Lon  DATE
37984    7900  3755  11231  1952
       Height   Lat    Lon  DATE
37985  102500  2715  11123  1952
       Hei

In [9]:
if __name__ == '__main__':
    for year in range(1962,1963):
        cf=df[df["DATE"]==str(year)]
        print(cf.head())
        stid=cf.Stid.drop_duplicates()
        print(stid)
        bf=[]
        al=[]
        for id in stid:
            #af=df[["Stid","rain","Height","Lat","Lon","DATE"]][df["Stid"]==id]
            af=cf[["Stid","rain"]][cf["Stid"]==id]
            ef=cf[["Height","Lat","Lon","DATE"]][cf["Stid"]==id]
            a=ef.iloc[[0]]
            #print(a)
        #af=cf[["Stid","rain"]][cf["Stid"]==id]
            if any(af["rain"]):#如果都为空、0、false，则返回false，如果不都为空、0、false，则返回true。
                bf.append(af)
                al.append(a)
                # TypeError: list indices must be integers or slices, not tuple 53602 is all 0.000
                     #for 1951Y bf[147] 56029 all is 66.0
        #print(len(bf))
    #print(bf)
        pool = mp.Pool(processes=8)
        multi_res =[pool.apply_async(job, (i,)) for i in range(len(bf))]
        pool.close()
        pool.join()#调用join之前，先调用close函数，否则会出错。执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    #name=['Stid','rain']
    #test=pd.DataFrame(columns=name,data=mres)#数据有三列，列名分别为one,two,three
    #test.to_csv('testcsv231.csv',index=False)
    #test.to_csv("ci_"+str(i)+".csv",index=False,sep=',')
        output= pd.DataFrame()
        for i in range(len(bf)):
            try:
                result0=multi_res[i].get()
                #print(result0)
        #cipd=pd.DataFrame(np.array([result0]),columns=["DATE","Stid","Height","Lat","Lon","ci"])
       # output.append(result0)
                output=pd.concat([output, result0], axis=0)
            except Exception as e:
                print("error")
        output.to_csv("ci_"+str(year)+".csv",index=False,sep=',')

Empty DataFrame
Columns: [Time, Stid, Lat, Lon, Height, rain, rain208, rain820, DATE]
Index: []
Series([], Name: Stid, dtype: int64)


if __name__ == '__main__':
    pool = mp.Pool(processes=8)
    multi_res =[pool.apply_async(job, (i,)) for i in range(len(bf))]

def multicore():
    pool = mp.Pool(processes=8)
    res = pool.map(job,range(len(bf)))
    res = pool.apply_async(job, (2,))
    print(res.get())
    multi_res =[pool.apply_async(job, (i,)) for i in range(len(bf))]
    print([res.get() for res in multi_res])

def multicore1():
    pool = mp.Pool(processes=8)
    #multi_res =[pool.apply_async(job, (231,))]
    res = pool.apply_async(job, (151,))
    #print(res.get())
    #print([res.get() for res in multi_res])
    #print(csv)
   #name=['Stid','rain']
    #test=pd.DataFrame(columns=name,data=csv)#数据有三列，列名分别为one,two,three
    #test.to_csv('testcsv231.csv',index=False)
    print(res.get())

## pd.pivot_table

cf=df[df["DATE"]=='1954Y']
table1 = pd.pivot_table(cf, values='rain', index=['Stid'],columns=['DATE'], aggfunc=ci_pre)
table1.to_csv("ci_1954.csv",index=True,sep=',')